In [1]:
import os
import numpy as np
import time
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as data
import torchvision
import torchvision.models as models
from torchvision import transforms
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score

import warnings
warnings.simplefilter("ignore")

In [2]:
MAX_ITER = 10
BATCH_SIZE = 40
LEARNING_RATE = 1e-3
TRAIN_DATA_PATH = "train/"
VAL_DATA_PATH = "val/"
NUM_CLASSES = 2

TRANSFORM = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ColorJitter(brightness=(0.8, 1.2), contrast=(1, 1.5), saturation=(1, 1.2)),
    transforms.ToTensor(),
    ])


train_data = torchvision.datasets.ImageFolder(root=TRAIN_DATA_PATH, transform=TRANSFORM)
train_data_loader = data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)
val_data = torchvision.datasets.ImageFolder(root=VAL_DATA_PATH, transform=TRANSFORM)
val_data_loader = data.DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
dataloaders = {'train': train_data_loader, 'val': val_data_loader}
dataset_sizes = {'train': len(train_data),'val':len(val_data)}

In [3]:
print(dataset_sizes)

{'train': 26430, 'val': 2981}


In [4]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_f1 = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                scheduler.step()
                model.train()  
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            epoch_pred = []
            epoch_truth = []

            batch = 0
            for inputs, labels in dataloaders[phase]:
                batch += 1
                inputs = inputs.to(device)
                labels = labels.to(device)
                # print(inputs.shape)
                # break

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):

                    if phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                        loss.backward()
                        optimizer.step()
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)
                    

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                epoch_truth.extend(labels.cpu().data)
                epoch_pred.extend(preds.cpu())
                
                if batch % 50 == 0 and phase == 'train':
                    print('Batch {} Loss {:.4f}'.format(batch, loss.item()))
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_f1 = f1_score(epoch_truth, epoch_pred, average='weighted')
            epoch_auc = roc_auc_score(epoch_truth, epoch_pred)
                          
            print('{} Loss: {:.4f} Acc: {:.4f} F1: {:.4f} AUC {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_f1,
                                                                            epoch_auc))
            
            if phase == 'val' and epoch_f1 > best_f1:
                best_f1 = epoch_f1
                best_model_wts = copy.deepcopy(model.state_dict())

        print('-'*5)

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best Val F1: {:4f}'.format(best_f1))

    model.load_state_dict(best_model_wts)
    return model

In [5]:
model_ft = models.inception_v3(pretrained=True)

# print(model_ft)
print(model_ft.fc)

Linear(in_features=2048, out_features=1000, bias=True)


In [6]:
num_ftrs = model_ft.AuxLogits.fc.in_features
model_ft.AuxLogits.fc = nn.Linear(num_ftrs, NUM_CLASSES)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs,NUM_CLASSES)

model_ft = model_ft.to(device)

w = [0.10598645840316419, 1.894013541596836]

weights = torch.tensor(w).to(device)

criterion = nn.CrossEntropyLoss(weight=weights)

optimizer_ft = optim.Adam(model_ft.parameters(), lr=LEARNING_RATE)

exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=2, gamma=0.5)

In [7]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=MAX_ITER)

Epoch 0/9
----------
Batch 50 Loss 0.8409
Batch 100 Loss 0.7189
Batch 150 Loss 0.7586
Batch 200 Loss 0.8850
Batch 250 Loss 0.9920
Batch 300 Loss 0.6323
Batch 350 Loss 0.5242
Batch 400 Loss 0.3079
Batch 450 Loss 0.6452
Batch 500 Loss 0.2865
Batch 550 Loss 0.6909
Batch 600 Loss 0.5049
Batch 650 Loss 1.4200
train Loss: 0.6437 Acc: 0.8773 F1: 0.9027 AUC 0.8018
val Loss: 0.4302 Acc: 0.9554 F1: 0.9570 AUC 0.8024
-----
Epoch 1/9
----------
Batch 50 Loss 0.6574
Batch 100 Loss 0.2953
Batch 150 Loss 0.6869
Batch 200 Loss 0.1758
Batch 250 Loss 0.2264
Batch 300 Loss 0.5584
Batch 350 Loss 0.2949
Batch 400 Loss 0.3762
Batch 450 Loss 0.1947
Batch 500 Loss 0.9833
Batch 550 Loss 0.4043
Batch 600 Loss 0.3893
Batch 650 Loss 0.1578
train Loss: 0.4014 Acc: 0.9265 F1: 0.9381 AUC 0.8879
val Loss: 0.3277 Acc: 0.9413 F1: 0.9487 AUC 0.8627
-----
Epoch 2/9
----------
Batch 50 Loss 0.1060
Batch 100 Loss 0.9383
Batch 150 Loss 0.1540
Batch 200 Loss 0.2336
Batch 250 Loss 0.0623
Batch 300 Loss 0.3083
Batch 350 Loss 0

In [ ]:
torch.save(model_ft.state_dict(), 'nova_ins.pth')

In [8]:
TEST_DATA_PATH = "test/"
WA_DATA_PATH = "test_wa/"
DEYE_DATA_PATH = "test_deye/"

test_data = torchvision.datasets.ImageFolder(root=TEST_DATA_PATH, transform=TRANSFORM)
test_data_loader  = data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

wa_data = torchvision.datasets.ImageFolder(root=WA_DATA_PATH, transform=TRANSFORM)
wa_data_loader = data.DataLoader(wa_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)

deye_data = torchvision.datasets.ImageFolder(root=DEYE_DATA_PATH, transform=TRANSFORM)
deye_data_loader = data.DataLoader(deye_data, batch_size=BATCH_SIZE, shuffle=True,  num_workers=4)


dataloaders = {'test_kaggle': test_data_loader, 'test_wa': wa_data_loader, 'test_deye': deye_data_loader}
dataset_sizes = {'test_kaggle': len(test_data),'test_wa':len(wa_data), 'test_deye':len(deye_data)}

In [9]:
print(dataset_sizes)

{'test_kaggle': 44836, 'test_wa': 45, 'test_deye': 38}


In [10]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

cnt = 6

def my_plot(truth, pred):
    auc = roc_auc_score(truth, pred)
    print('AUC: %.4f' % auc)
    fpr, tpr, thresholds = roc_curve(truth, pred)
    plt.plot([0, 1], [0, 1], linestyle='--')
    plt.plot(fpr, tpr, marker='.')
    plt.xlabel('Specificity / True Negative Rate')
    plt.ylabel('Sensitivity / Recall')
    plt.savefig(str(cnt) + ".jpg")
    cnt += 1
    plt.show()

In [13]:
def eval_model(model, criterion, optimizer, scheduler):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    best_f1 = 0.0

    for epoch in range(1):

        # Each epoch has a training and validation phase
        for phase in ['test_kaggle', 'test_wa', 'test_deye']:
        # for phase in ['test_wa', 'test_deye']:
            model.eval()   # Set model to evaluate mode
            print('Evaluating {}'.format(phase))
            running_loss = 0.0
            running_corrects = 0
            epoch_pred = []
            epoch_truth = []

            # Iterate over data.
            batch = 0
            # batch_start = time.time()
            for inputs, labels in dataloaders[phase]:
                batch += 1
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    if phase == 'train':
                        outputs, aux_outputs = model(inputs)
                        _, preds = torch.max(outputs, 1)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                        loss.backward()
                        optimizer.step()
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)
                        _, preds = torch.max(outputs, 1)

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                epoch_truth.extend(labels.cpu().data)
                epoch_pred.extend(preds.cpu())

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            epoch_f1 = f1_score(epoch_truth, epoch_pred, average='weighted')
                          
            print('{} Loss: {:.4f} Acc: {:.4f} F1: {:.4f}'.format(phase, epoch_loss, epoch_acc, epoch_f1))
            print(classification_report(epoch_truth, epoch_pred))
            print(confusion_matrix(epoch_truth, epoch_pred))
            print(roc_auc_score(epoch_truth, epoch_pred))
            # my_plot(epoch_truth, epoch_pred)
            
        print('-'*5)
    return

In [14]:
eval_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler)

Evaluating test_wa
test_wa Loss: 9.2468 Acc: 0.1111 F1: 0.2000
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.11      0.20        45

   micro avg       0.11      0.11      0.11        45
   macro avg       0.50      0.06      0.10        45
weighted avg       1.00      0.11      0.20        45

[[ 0  0]
 [40  5]]
Evaluating test_deye
test_deye Loss: 1.3113 Acc: 0.4474 F1: 0.4415
              precision    recall  f1-score   support

           0       0.40      0.33      0.36        18
           1       0.48      0.55      0.51        20

   micro avg       0.45      0.45      0.45        38
   macro avg       0.44      0.44      0.44        38
weighted avg       0.44      0.45      0.44        38

[[ 6 12]
 [ 9 11]]
-----


NameError: name 'torch' is not defined